# Exploring Why Civil Resistance Works
---

> Gaganis Michail, Pregraduate Student <br />
> Department of Management Science and Technology <br />
> Athens University of Economics and Business <br />
> t8200234@aueb.gr

* This presentation contains a study on the Civil Resistance movements across the world from 1900-onwards.

* This study takes inspiration from the book [Why Civil Resistance Works](https://www.ericachenoweth.com/research/wcrw) by [Erica Chenoweth](https://www.ericachenoweth.com/) and Maria J. Stephan.

* The research done in this repository is done in the in the context of the *Applied Machine Learning* course, taught by *Mr. Panos Louridas* at the *Department of Management Science and Technology, AUEB*. The assignment can be found [here](/assignment/wcrw_v2.ipynb).

* The study takes place in November-December of `2023`, and references data up until that point in time.

# Dependencies for this Notebook

* In this Jupyter Notebook we shall use Python modules that the reader may or may not have installed. 
* The following code block shall install those for them so that no functionality problems arise due to missing modules.

* Links to documentation for all modules used can be found below:<br/>
-[Numpy](https://numpy.org/)<br/>
-[Scikit-learn](https://scikit-learn.org/stable/)<br/>
-[MatPlotLib](https://matplotlib.org/)<br/>
-[Pandas](https://pandas.pydata.org/)

In [2]:
!pip install numpy
!pip install scikit-learn
!pip install matplotlib
!pip install pandas

* We should go ahead and import the modules as well.

In [1]:
import numpy as np
import matplotlib as plt
import pandas as pd

# Data Used 
* For our research we will draw data from the [Nonviolent and Violent Campaigns and Outcomes (NAVCO) Dataset](https://dataverse.harvard.edu/dataverse/navco), and in particular the [NAVCO 1.2 Dataset](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/0UZOTX). In the repository is included the [NAVCO Data dictionary](data/NAVCO_data_dictionary.pdf) that provides more clarity as to what the different variables measure.

* We shall also use data from the [Polity Project](https://www.systemicpeace.org/polityproject.html), available from [Integrated Network for Societal Conflict Research (INSCR)](https://www.systemicpeace.org/inscrdata.html), in particular the [Polity5 Annual Time-Series, 1946-2018](http://www.systemicpeace.org/inscr/p5v2018.xls). The [Polity5 Data Dictionary](data/Polity5DataDictionary.pdf) is provided as well for all intensive purposes.

* We load our data in *Pandas Dataframes* so we can use them for analysis.

In [3]:
navco_data = pd.read_csv('data/NAVCO_data.csv')
navco_data.head(5)

,NAVCO ID,CAMPAIGN,LOCATION,BYEAR,EYEAR,TARGET,SUCCESS,LIMITED,FAILURE,ONGOING,...,OTHER,DURATION,PARTICIPATION,PERCENTAGE POPULAR PARTICIPATION,REGVIOL,DEFECT,VIOLENT FLANK,VIOLSIM,REGAID,STATESUP
0,4.0,Afghans,Afghanistan,1978.0,1979.0,Afghan government,1.0,0.0,0.0,0.0,...,0.0,613.0,10000.0,0.0006,1.0,1.0,NaN,1.0,1.0,1.0
1,3.0,Afghan resistance,Afghanistan,1980.0,1989.0,Soviet occupation,1.0,0.0,0.0,0.0,...,0.0,3281.0,10000.0,0.0007,1.0,0.0,NaN,1.0,0.0,1.0
2,218.0,Taliban/anti-government forces,Afghanistan,1992.0,1996.0,Afghan regime,1.0,0.0,0.0,0.0,...,0.0,1370.0,20000.0,0.0009,1.0,0.0,NaN,1.0,0.0,0.0
3,217.0,Taliban,Afghanistan,2001.0,2014.0,Afghan government,0.0,0.0,1.0,1.0,...,0.0,4391.0,60000.0,0.0016,1.0,1.0,NaN,1.0,1.0,0.0
4,5.0,Albanian anti-communist,Albania,1990.0,1991.0,Communist regime,1.0,0.0,0.0,0.0,...,0.0,442.0,300000.0,0.0911,1.0,0.0,0.0,1.0,1.0,0.0


In [4]:
polity_data = pd.read_csv('data/polity_data.csv')
polity_data.head(5)

,p5,cyear,ccode,scode,country,year,flag,fragment,democ,autoc,...,interim,bmonth,bday,byear,bprec,post,change,d5,sf,regtrans
0,0,7001800,700,AFG,Afghanistan,1800,0,NaN,1,7,...,NaN,1.0,1.0,1800.0,1.0,-6.0,88.0,1.0,NaN,NaN
1,0,7001801,700,AFG,Afghanistan,1801,0,NaN,1,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,7001802,700,AFG,Afghanistan,1802,0,NaN,1,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,7001803,700,AFG,Afghanistan,1803,0,NaN,1,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,7001804,700,AFG,Afghanistan,1804,0,NaN,1,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Introduction

* Erica Chenoweth and Maria J. Stephan published a book in 2011 called *Why Civil Resistance Works: The Strategic Logic of Nonviolent Conflict*. 

* The book contains an in-depth analysis of many Civil Resistance movements through 1900 to 2006, and seeks to compare the success rate of violent and non-violent movements.
* The dataset used to draw conclusions for the book is public and used in this notebook.
* This presentation is complementary to the book as it seeks to cross-validate the findings noted, as well as dig deeper and draw its own cocnlusions.